## Web Scraping

In [123]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [129]:
url = "https://www.airlinequality.com/airline-reviews/flight/page/page_number/?sortby=post_date%3ADesc&pagesize=100"


dfs = []
airlines = ['ryanair', 'delta-air-lines', 'spirit-airlines', 'united-airlines', 'emirates', 'qatar-airways', 'etihad-airways']
 
for airline in airlines:
    urllink = url.replace('flight', airline)
    for i in range (1,20):
        
        response = requests.get(urllink.replace('page_number', str(i)))
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            reviews = soup.find_all("article", itemprop ="review")

            for item in reviews:
                my_dict = {
                                "rating": "",
                                "country": "",
                                "date": "",
                                "review": "",
                                "Aircraft": '',
                                "Type Of Traveller": '',
                                "Seat Type": '',
                                "Route": '',
                                "Date Flown": '',
                                "Seat Comfort": '',
                                "Cabin Staff Service": '',
                                "Food & Beverages": '',
                                "Inflight Entertainment": '',
                                "Ground Service": '',
                                "Wifi & Connectivity": '',
                                "Value For Money": '',
                                "Recommended": '',
                                "Airlines": airline
                                }
                
                try:
                    my_dict['rating'] = item.find("span", itemprop="ratingValue").text.strip()
                except AttributeError:
                    pass  

                try:
                    my_dict['review'] = item.find("div", class_="text_content").text.strip()
                except AttributeError:
                    pass 
                
                try:
                    temp = item.find("h3", class_="text_sub_header").text.strip()
                    my_dict['author'] = temp.split('(')[0].strip()
                    my_dict['country'] = temp.split('(')[1].split(')')[0].strip()
                    my_dict['date'] = temp.split(')')[1].strip()
                except AttributeError:
                    pass
                
                my_dict['Aircraft'] = item.find("td", class_ ='')

                for i in item.find_all('td', 'review-value'):
                    my_dict[i.previous_sibling.get_text().strip()] = i.get_text().strip()

                for j in item.find_all('td', 'review-rating-stars'):
                    category = j.parent.find('td', 'review-rating-header').text.strip()
                    my_dict[category] = str(len(j.parent.find_all('span', class_="fill")))

                dfs.append(pd.DataFrame([my_dict]))


output = pd.concat(dfs, ignore_index=True)


In [130]:
output.head()

,rating,country,date,review,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Ground Service,Wifi & Connectivity,Value For Money,Recommended,Airlines,author
0,3,Italy,1st April 2024,Not Verified | Very cheeky check-in system: t...,None,Couple Leisure,Economy Class,Manchester to Milan,March 2024,1,2,,1,2,,2,no,ryanair,Y Chen
1,1,Spain,28th March 2024,Not Verified | Terrible customer service. Han...,A321,Family Leisure,Economy Class,Marrakech to Sevilla,March 2024,2,4,,1,1,1,3,no,ryanair,Diego Perez
2,5,United Kingdom,18th March 2024,✅ Trip Verified | Luton to Faro and return Fa...,None,Couple Leisure,Economy Class,Luton to Faro,March 2024,1,3,,,1,,2,no,ryanair,15 reviews\n\n\n\nRichard Hodges
3,1,Canada,16th March 2024,✅ Trip Verified | Very bad airline. I spent 1...,None,Family Leisure,Economy Class,Edinburgh to Stansted,March 2024,1,1,,,1,,2,no,ryanair,G Han
4,7,United Kingdom,10th March 2024,✅ Trip Verified | When things go wrong with l...,737-8200,Solo Leisure,Economy Class,Stansted to Sofia,March 2024,3,4,2,,3,,5,yes,ryanair,R Vines


In [133]:
output.to_csv('data_csv.csv', index=False)
output.to_excel('data_excel.xlsx', index=False)

In [132]:
import sqlite3
conn = sqlite3.connect('data.db')
output.to_sql('data', conn, index=False, if_exists='replace')
conn.close()


 # Exploration

In [131]:
df.Recommended.value_counts()

no     8772
yes    4387
Name: Recommended, dtype: int64

In [124]:
df = output.replace(r'^\s*$', np.nan, regex=True)

In [126]:
df.isna().sum()

rating                      46
country                      0
date                         0
review                       0
Aircraft                  8614
Type Of Traveller          886
Seat Type                    0
Route                      960
Date Flown                 892
Seat Comfort               757
Cabin Staff Service        801
Food & Beverages          3390
Inflight Entertainment    4535
Ground Service            1310
Wifi & Connectivity       7263
Value For Money              2
Recommended                  0
author                       0
dtype: int64

1. We can drop 'Wifi & Connectivity' and  'Aircraft'

2. Impute NaN with median for rating, Mean for Seat Comfort, Cabin Staff Service, Food & Beverages, Inflight Entertainment, Ground Service, Wifi & Connectivity, Value For Money. Mode for Country, Seat Type

3. Create new category - Unknown for Food & Beverages, Inflight Entertainment, Ground Service etc